In [1]:
!pip install wordcloud
!pip install contractions
!pip install gensim
!pip install xgboost
!pip install pyLDAvis

In [2]:
import pandas as pd
data = '../Downloads/train.csv'
df = pd.read_csv(data)
print(df.columns)
print(df["Score"].value_counts())

Index(['Score', 'Text'], dtype='object')
Score
5    197151
4     43876
1     28521
3     23296
2     16287
Name: count, dtype: int64


In [3]:
df['Text'] = df['Text'].str.lower()
df.head()

,Score,Text
0,5,i received this product early from the seller!...
1,5,*****<br />numi's collection assortment melang...
2,5,"i was very careful not to overcook this pasta,..."
3,5,buying this multi-pack i was misled by the pic...
4,5,these bars are so good! i loved them warmed up...


In [4]:
import string
df['Text'] = df['Text'].str.translate(str.maketrans('', '', string.punctuation))
df.head()

,Score,Text
0,5,i received this product early from the seller ...
1,5,br numis collection assortment melange include...
2,5,i was very careful not to overcook this pasta ...
3,5,buying this multipack i was misled by the pict...
4,5,these bars are so good i loved them warmed up ...


In [5]:
df['Text'] = df['Text'].str.replace(r'\d+', '', regex=True)

In [6]:
import nltk
from nltk.corpus import stopwords
import contractions

nltk.download('stopwords')

def expand_and_remove_stopwords(text):
    expanded_text = contractions.fix(text)
    stop_words = set(stopwords.words('english'))
    words = expanded_text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

df['Text'] = df['Text'].apply(expand_and_remove_stopwords)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/geraint/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Score,Text
0,5,received product early seller tastey great mid...
1,5,br numis collection assortment melange include...
2,5,careful overcook pasta making sure take bite e...
3,5,buying multipack misled picture whole hazel nu...
4,5,bars good loved warmed definitely think great ...


In [7]:
import nltk
from nltk.corpus import stopwords
import contractions
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
df['TextTokens'] = df['Text'].apply(word_tokenize)

df.head()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/geraint/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,Score,Text,TextTokens
0,5,received product early seller tastey great mid...,"[received, product, early, seller, tastey, gre..."
1,5,br numis collection assortment melange include...,"[br, numis, collection, assortment, melange, i..."
2,5,careful overcook pasta making sure take bite e...,"[careful, overcook, pasta, making, sure, take,..."
3,5,buying multipack misled picture whole hazel nu...,"[buying, multipack, misled, picture, whole, ha..."
4,5,bars good loved warmed definitely think great ...,"[bars, good, loved, warmed, definitely, think,..."


In [8]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
df['TextTokens'] = df['TextTokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['TextTokens'] = df['TextTokens'].apply(lambda tokens: ' '.join(tokens))
df.head()

[nltk_data] Downloading package wordnet to /Users/geraint/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Score,Text,TextTokens
0,5,received product early seller tastey great mid...,received product early seller tastey great mid...
1,5,br numis collection assortment melange include...,br numis collection assortment melange include...
2,5,careful overcook pasta making sure take bite e...,careful overcook pasta making sure take bite e...
3,5,buying multipack misled picture whole hazel nu...,buying multipack misled picture whole hazel nu...
4,5,bars good loved warmed definitely think great ...,bar good loved warmed definitely think great s...


In [9]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
import gensim.downloader as api
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [10]:
X = df['TextTokens']
y = df['Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2))

In [12]:
X_train_tfidf = tfidf.fit_transform(X_train)

In [13]:
model = RandomForestClassifier(n_estimators=200, class_weight="balanced", random_state=42)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=200,
                       random_state=42)

In [14]:
X_test_tfidf = tfidf.transform(X_test)

In [15]:
y = model.predict(X_test_tfidf) 

In [16]:
model.score(X_test_tfidf,y_test)

0.6774871819755123

In [17]:
model.score(X_train_tfidf,y_train)

0.9997654708375118

In [65]:
testData = '../Downloads/test.csv'
test = pd.read_csv(testData)

In [66]:
test.head()

,Id,Text
0,0,Fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just..."
2,2,Not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...
4,4,"<a href=""http://www.amazon.com/gp/product/B000..."


In [69]:
test['Text'] = test['Text'].str.lower()
test.head()

,Id,Text
0,0,fog chaser was the best both in flavor and bod...
1,1,"we found this at pf changs, and it tastes just..."
2,2,not quite a chocolate bar substitute but delic...
3,3,this is not as tasty as pamela's almond biscot...
4,4,"<a href=""http://www.amazon.com/gp/product/b000..."


In [71]:
import pandas as pd
import re

def clean_text_with_html_urls(df, text_column='Text'):
    """
    Cleans text within a pandas DataFrame column, removing HTML anchor tags and URLs,
    and converting the text to lowercase.

    Args:
        df (pd.DataFrame): The DataFrame containing the text data.
        text_column (str): The name of the column containing the text.

    Returns:
        pd.DataFrame: The DataFrame with cleaned text.
    """

    def remove_html_urls(text):
        """Removes HTML anchor tags containing URLs."""
        html_url_pattern = re.compile(r'<a\s+href="http[^"]*">')
        return html_url_pattern.sub(r'', text)

    def remove_url_remainder(text):
        """Removes the remainder of the anchor tag."""
        remainder_pattern = re.compile(r'</a>')
        return remainder_pattern.sub(r'', text)

    def remove_all_urls(text):
        """Combines to remove both the anchor tag and the url"""
        text = remove_html_urls(text)
        text = remove_url_remainder(text)
        return text

    df[text_column] = df[text_column].astype(str).apply(remove_all_urls) #handle nan and other types
    df[text_column] = df[text_column].str.lower()
    return df

In [73]:
test = clean_text_with_html_urls(test)
test.head()

,Id,Text
0,0,fog chaser was the best both in flavor and bod...
1,1,"we found this at pf changs, and it tastes just..."
2,2,not quite a chocolate bar substitute but delic...
3,3,this is not as tasty as pamela's almond biscot...
4,4,old mother hubbard special recipe mini puppy b...


In [75]:
test['Text'] = test['Text'].str.translate(str.maketrans('', '', string.punctuation))
test.head()

,Id,Text
0,0,fog chaser was the best both in flavor and bod...
1,1,we found this at pf changs and it tastes just ...
2,2,not quite a chocolate bar substitute but delic...
3,3,this is not as tasty as pamelas almond biscott...
4,4,old mother hubbard special recipe mini puppy b...


In [77]:
test['Text'] = test['Text'].str.replace(r'\d+', '', regex=True)

In [79]:
test['Text'] = test['Text'].apply(expand_and_remove_stopwords)
test.head()

,Id,Text
0,0,fog chaser best flavor body far product body c...
1,1,found pf changs tastes great bags per box listed
2,2,quite chocolate bar substitute delicious anywa...
3,3,tasty pamelas almond biscotti good chocolate b...
4,4,old mother hubbard special recipe mini puppy b...


In [81]:
test['TextTokens'] = test['Text'].apply(word_tokenize)
test.head()

,Id,Text,TextTokens
0,0,fog chaser best flavor body far product body c...,"[fog, chaser, best, flavor, body, far, product..."
1,1,found pf changs tastes great bags per box listed,"[found, pf, changs, tastes, great, bags, per, ..."
2,2,quite chocolate bar substitute delicious anywa...,"[quite, chocolate, bar, substitute, delicious,..."
3,3,tasty pamelas almond biscotti good chocolate b...,"[tasty, pamelas, almond, biscotti, good, choco..."
4,4,old mother hubbard special recipe mini puppy b...,"[old, mother, hubbard, special, recipe, mini, ..."


In [83]:
test['TextTokens'] = test['TextTokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
test['TextTokens'] = test['TextTokens'].apply(lambda tokens: ' '.join(tokens))
test.head()

,Id,Text,TextTokens
0,0,fog chaser best flavor body far product body c...,fog chaser best flavor body far product body c...
1,1,found pf changs tastes great bags per box listed,found pf chang taste great bag per box listed
2,2,quite chocolate bar substitute delicious anywa...,quite chocolate bar substitute delicious anywa...
3,3,tasty pamelas almond biscotti good chocolate b...,tasty pamelas almond biscotti good chocolate b...
4,4,old mother hubbard special recipe mini puppy b...,old mother hubbard special recipe mini puppy b...


In [85]:
x_test = tfidf.fit_transform(test['Text'])

In [87]:
yy=model.predict(x_test) 

In [88]:
yy.shape

(119662,)

In [91]:
sample=pd.read_csv('../Downloads/SampleSubmission.csv') # submission of majority class 

In [93]:
sample.head()

,Id,Score
0,0,3
1,1,3
2,2,3
3,3,3
4,4,3


In [95]:
sample['Score'] = yy # save the labels for your model to csv file, you will use this for you Kaggle competition submission
sample.to_csv('Testsubmission.csv', index=False)

In [97]:
view_submitted=pd.read_csv('Testsubmission.csv')

In [99]:
view_submitted.describe()

,Id,Score
count,119662.000000,119662.000000
mean,59830.500000,4.935744
std,34543.588293,0.497501
min,0.000000,1.000000
25%,29915.250000,5.000000
50%,59830.500000,5.000000
75%,89745.750000,5.000000
max,119661.000000,5.000000


In [101]:
view_submitted.head()

,Id,Score
0,0,5
1,1,5
2,2,5
3,3,5
4,4,5
